## 0. Setup and Dependencies

In [22]:
# Import required libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import json
from pathlib import Path
from typing import List, Dict, Optional, Tuple
from datetime import datetime
from queue import Queue
import concurrent.futures
import threading

print("✓ Libraries imported successfully")


✓ Libraries imported successfully


## 1. Data Cleaning Functions

In [23]:
def clean_integer(value):
    """
    Convert value to integer safely.
    If 'Khác', garbage text, or null -> return -1
    Examples: "5 chỗ" -> 5, "7 cửa" -> 7
    """
    if pd.isna(value):
        return -1
    
    val_str = str(value).strip()
    
    # If already a number
    if val_str.isdigit() or (val_str.startswith('-') and val_str[1:].isdigit()):
        return int(val_str)
        
    # Handle cases like "5 chỗ", "5 cửa" -> Extract number 5
    # Use regex to keep only digits
    nums = re.findall(r'\d+', val_str)
    if nums:
        return int(nums[0])
        
    # Other cases (e.g., 'Khác') -> -1
    return -1

def clean_float(value):
    """
    Convert to float (for engine capacity).
    Examples: "2.0 L" -> 2.0, "1.5" -> 1.5
    """
    if pd.isna(value):
        return -1.0
    try:
        # Handle "2.0 L" -> 2.0
        val_str = str(value).upper().replace('L', '').replace(',', '').strip()
        return float(val_str)
    except:
        return -1.0

def clean_string(value):
    """
    Clean string values, convert -1 or empty to -1
    """
    if pd.isna(value):
        return -1
    
    val_str = str(value).strip()
    
    # If empty or "-1" string
    if val_str == '' or val_str == '-1':
        return -1
        
    return val_str

def clean_car_data(car_data: Dict) -> Dict:
    """
    Clean and format car data before saving to CSV.
    Applies all cleaning functions to ensure data consistency.
    """
    cleaned = {
        'url': str(car_data.get('url', -1)),
        'price': clean_integer(car_data.get('price', -1)),
        'brand': clean_string(car_data.get('brand', -1)),
        'model': clean_string(car_data.get('model', -1)),
        'year': clean_integer(car_data.get('year', -1)),
        'odometer': clean_integer(car_data.get('odometer', -1)),
        'transmission': clean_string(car_data.get('transmission', -1)),
        'fuel_type': clean_string(car_data.get('fuel_type', -1)),
        'engine_capacity': clean_float(car_data.get('engine_capacity', -1)),
        'body_style': clean_string(car_data.get('body_style', -1)),
        'origin': clean_string(car_data.get('origin', -1)),
        'seats': clean_integer(car_data.get('seats', -1)),
        'condition': clean_string(car_data.get('condition', -1)),
        'num_owners': clean_integer(car_data.get('num_owners', -1)),
        'inspection_status': clean_integer(car_data.get('inspection_status', -1)),
        'warranty_status': clean_integer(car_data.get('warranty_status', -1)),
        'source': clean_string(car_data.get('source', -1))
    }
    
    return cleaned

print("✓ Data cleaning functions defined")

✓ Data cleaning functions defined


## 2. Configuration and Constants

In [24]:
# HTTP Headers
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Car brands list (113 brands)
CAR_BRANDS = [
    "toyota", "hyundai", "kia", "ford", "honda", 
    "mazda", "mitsubishi", "vinfast", "chevrolet", 
    "nissan", "suzuki",
    
    "mercedes_benz", "bmw", "audi", "lexus", 
    "landrover", "porsche", "volvo", "peugeot", 
    "mini", "subaru", "volkswagen",
    
    "daewoo", 

    "isuzu", "jeep",

    "mg", "baic", "wuling", "byd" 
]

# Brand mapping (URL code -> Display name)
BRAND_MAPPING = {
    "toyota": "Toyota",
    "honda": "Honda",
    "mazda": "Mazda",
    "mitsubishi": "Mitsubishi",
    "nissan": "Nissan",
    "suzuki": "Suzuki",
    "subaru": "Subaru",
    "isuzu": "Isuzu",
    "lexus": "Lexus",
    
    "hyundai": "Hyundai",
    "kia": "Kia",
    "daewoo": "Daewoo",
    "ssangyong": "Ssangyong",
    
    "ford": "Ford",
    "chevrolet": "Chevrolet",
    "jeep": "Jeep",
    
    "mercedes_benz": "Mercedes-Benz",
    "bmw": "BMW",
    "audi": "Audi",
    "volkswagen": "Volkswagen",
    "porsche": "Porsche",
    "peugeot": "Peugeot",
    "landrover": "Land Rover",
    "volvo": "Volvo",
    "mini": "MINI",
    
    "vinfast": "VinFast",
    
    "mg": "MG",
    "baic": "BAIC",
    "byd": "BYD",
    "wuling": "Wuling"
}

print(f"✓ Total brands available: {len(CAR_BRANDS)}")

✓ Total brands available: 29


## 3. Helper Functions

In [25]:
def rate_limit(delay: float = 0.0):
    """Add delay between requests."""
    time.sleep(delay)

def get_headers() -> dict:
    """Get HTTP headers for requests."""
    return HEADERS.copy()

def split_price(title: str) -> int:
    """
    Extract price from Vietnamese title with Tỷ and Triệu.
    Example: '2 Tỷ 500 Triệu' -> 2500000000 VND
    """
    match_ty = re.search(r'(\d+)\s*Tỷ', title, re.IGNORECASE)
    ty = int(match_ty.group(1)) if match_ty else 0
    
    match_trieu = re.search(r'(\d+)\s*Triệu', title, re.IGNORECASE)
    trieu = int(match_trieu.group(1)) if match_trieu else 0
    
    price = (ty * 1_000_000_000) + (trieu * 1_000_000)
    return price

def extract_model_from_title(title: str, brand: str) -> Tuple[str, str]:
    """
    Extract car model and year from title.
    Returns: (model, year)
    """
    # Remove price information
    price_pattern = r'\d+\s*Tỷ|\d+\s*Triệu'
    price_match = re.search(price_pattern, title)
    title_without_price = title[:price_match.start()].strip() if price_match else title
    
    # Remove brand name
    brand_display = BRAND_MAPPING.get(brand, brand).replace('_', ' ')
    title_lower = title_without_price.lower()
    brand_lower = brand_display.lower()
    
    if title_lower.startswith(brand_lower):
        remaining = title_without_price[len(brand_display):].strip()
    else:
        remaining = title_without_price
    
    # Extract year
    year_pattern = r'\b(19|20)\d{2}\b'
    year_match = re.search(year_pattern, remaining)
    
    if year_match:
        model = remaining[:year_match.start()].strip()
        year = year_match.group()
    else:
        words = remaining.split()
        model = ' '.join(words[:3]) if len(words) >= 3 else remaining
        year = None
    
    return model, year

def extract_engine_info(text: str) -> Tuple[str, str]:
    """
    Extract fuel type and engine capacity.
    Example: 'Xăng 2.0 L' -> ('Xăng', '2.0')
    """
    if ":" in text:
        value_part = text.split(":", 1)[1].strip()
    else:
        value_part = text.strip()
    
    match = re.search(r'^(.*?)\s+(\d+(?:\.\d+)?)\s*L$', value_part, re.IGNORECASE)
    
    if match:
        return match.group(1).strip(), match.group(2).strip()
    else:
        return value_part, None

print("✓ Helper functions defined")

✓ Helper functions defined


## 4. Parsing Functions

In [26]:
def parse_bonbanh_car_details(soup: BeautifulSoup, brand: str = None) -> Dict[str, str]:
    """
    Parse car details from Bonbanh.com page.
    Returns structured dictionary with 15 fields.
    """
    car_data = {
        'price': -1, 'brand': -1, 'model': -1, 'year': -1, 'odometer': -1,
        'transmission': -1, 'fuel_type': -1, 'engine_capacity': -1,
        'body_style': -1, 'origin': -1, 'seats': -1, 'condition': -1,
        'num_owners': -1, 'inspection_status': -1, 'warranty_status': -1,
        'source': 'bonbanh'
    }
    
    try:
        # Extract title and price
        title_div = soup.find('div', class_='title')
        if title_div and title_div.find('h1'):
            title = title_div.find('h1').text
            title = re.sub(r'\s+', ' ', title).strip()
            
            price = split_price(title)
            car_data['price'] = price if price else -1
            
            if brand:
                car_data['brand'] = BRAND_MAPPING.get(brand, brand)
                model, year = extract_model_from_title(title, brand)
                car_data['model'] = model if model else -1
                car_data['year'] = year if year else -1
        
        # Extract details
        detail_div = soup.find('div', class_='box_car_detail')
        if detail_div:
            mail_parents = detail_div.find_all('div', id='mail_parent')
            
            for mail_parent in mail_parents:
                label_elem = mail_parent.find('label')
                span_elem = mail_parent.find('span')
                
                if not label_elem or not span_elem:
                    continue
                
                label = re.sub(r'\s+', ' ', label_elem.text).strip().rstrip(':')
                value = re.sub(r'\s+', ' ', span_elem.text).strip()
                
                # Map labels to fields
                if label == 'Số Km đã đi':
                    car_data['odometer'] = re.sub(r'\D', '', value)
                elif label == 'Hộp số':
                    car_data['transmission'] = value
                elif label == 'Động cơ':
                    fuel_type, engine_capacity = extract_engine_info(value)
                    car_data['fuel_type'] = fuel_type if fuel_type else -1
                    car_data['engine_capacity'] = engine_capacity if engine_capacity else -1
                elif label == 'Kiểu dáng':
                    car_data['body_style'] = value
                elif label == 'Xuất xứ':
                    car_data['origin'] = value
                elif label == 'Số chỗ ngồi':
                    car_data['seats'] = value
                elif label == 'Tình trạng':
                    car_data['condition'] = value
                elif 'chủ' in label.lower():
                    car_data['num_owners'] = value
    
    except Exception as e:
        print(f"Error parsing: {e}")
    
    return car_data

def parse_bonbanh_listing(soup: BeautifulSoup) -> List[str]:
    """Extract car detail URLs from Bonbanh listing page."""
    car_links = []
    try:
        car_lis = soup.find_all('li', class_='car-item')
        for car_li in car_lis:
            a_link = car_li.find('a')
            if a_link and 'href' in a_link.attrs:
                link = 'https://bonbanh.com/' + a_link.attrs['href']
                car_links.append(link)
    except Exception as e:
        print(f"Error parsing listing: {e}")
    return car_links

def parse_chotot_car_details(soup: BeautifulSoup) -> Dict[str, str]:
    """Parse car details from Chotot.com page."""
    car_data = {
        'price': -1, 'brand': -1, 'model': -1, 'year': -1, 'odometer': -1,
        'transmission': -1, 'fuel_type': -1, 'engine_capacity': -1,
        'body_style': -1, 'origin': -1, 'seats': -1, 'condition': -1,
        'num_owners': -1, 'inspection_status': -1, 'warranty_status': -1,
        'source': 'chotot'
    }
    
    try:
        # Extract price
        div_title = soup.find('div', class_='cpmughi')
        if div_title:
            price_elem = div_title.find('b', class_='p26z2wb')
            if price_elem:
                price = re.sub(r'\D', '', price_elem.text.strip())
                car_data['price'] = price if price else -1
        
        # Extract other details
        div_list = soup.find_all('div', class_='p1ja3eq0')
        for div in div_list:
            label_elem = div.find('span', class_='bwq0cbs')
            if label_elem:
                value_elem = label_elem.find_next_sibling('span')
                if value_elem:
                    label = label_elem.text.strip()
                    value = value_elem.text.strip()
                    
                    # Map labels
                    if label == 'Số Km đã đi': car_data['odometer'] = value
                    elif label == 'Hộp số': car_data['transmission'] = value
                    elif label == 'Nhiên liệu': car_data['fuel_type'] = value
                    elif label == 'Kiểu dáng': car_data['body_style'] = value
                    elif label == 'Xuất xứ': car_data['origin'] = value
                    elif label == 'Số chỗ': car_data['seats'] = value
                    elif label == 'Tình trạng': car_data['condition'] = value
                    elif label == 'Hãng': car_data['brand'] = value
                    elif label == 'Năm sản xuất': car_data['year'] = value
                    elif label == 'Dòng xe': car_data['model'] = value
    
    except Exception as e:
        print(f"Error parsing: {e}")
    
    return car_data

def parse_chotot_listing(soup: BeautifulSoup) -> List[str]:
    """Extract car detail URLs from Chotot listing page."""
    car_links = []
    base_url = 'https://xe.chotot.com'
    try:
        a_list = soup.find_all('a', class_='c15fd2pn')
        for a in a_list:
            if 'href' in a.attrs:
                link = a.attrs['href']
                if not link.startswith('http'):
                    link = base_url + link
                car_links.append(link)
    except Exception as e:
        print(f"Error parsing listing: {e}")
    return car_links

print("✓ Parsing functions defined")

✓ Parsing functions defined


## 5. Progress Tracker Class

In [27]:
class ProgressTracker:
    """Track crawling progress to enable resume functionality."""
    
    def __init__(self, progress_file: str = "../data/crawl_progress.json"):
        self.progress_file = Path(progress_file)
        self.progress_file.parent.mkdir(parents=True, exist_ok=True)
        self.lock = threading.Lock()
        self.progress_data = self._load_progress()
    
    def _load_progress(self) -> Dict:
        if self.progress_file.exists():
            try:
                with open(self.progress_file, 'r', encoding='utf-8') as f:
                    return json.load(f)
            except:
                return {}
        return {}
    
    def _save_progress(self):
        try:
            with open(self.progress_file, 'w', encoding='utf-8') as f:
                json.dump(self.progress_data, f, indent=2, ensure_ascii=False)
        except Exception as e:
            print(f"Warning: Could not save progress: {e}")
    
    def get_last_page(self, source: str, brand: str = None) -> int:
        key = f"{source}_{brand}" if brand else source
        return self.progress_data.get(key, {}).get('last_page', 0)
    
    def update_page(self, source: str, page: int, brand: str = None):
        with self.lock:
            key = f"{source}_{brand}" if brand else source
            if key not in self.progress_data:
                self.progress_data[key] = {}
            self.progress_data[key]['last_page'] = page
            self._save_progress()
    
    def mark_completed(self, source: str, brand: str = None):
        with self.lock:
            key = f"{source}_{brand}" if brand else source
            if key in self.progress_data:
                self.progress_data[key]['completed'] = True
                self._save_progress()
    
    def is_completed(self, source: str, brand: str = None) -> bool:
        key = f"{source}_{brand}" if brand else source
        return self.progress_data.get(key, {}).get('completed', False)
    
    def reset(self, source: str = None, brand: str = None):
        with self.lock:
            if source is None:
                self.progress_data = {}
            else:
                key = f"{source}_{brand}" if brand else source
                if key in self.progress_data:
                    del self.progress_data[key]
            self._save_progress()

print("✓ ProgressTracker class defined")

✓ ProgressTracker class defined


## 6. Bonbanh Crawler

In [28]:
class BonbanhCrawler:
    """Crawler for Bonbanh.com car listings."""
    
    BASE_URL = "https://bonbanh.com/oto/"
    
    def __init__(self, delay: float = 1.0):
        self.headers = get_headers()
        self.delay = delay
        self.current_brand = None
        self.csv_lock = threading.Lock()
        self.progress = ProgressTracker()
        self.existing_urls = set()  # Track URLs to prevent duplicates
    
    def crawl_car_details(self, link: str, brand: str = None) -> Optional[Dict[str, str]]:
        """Crawl details of a single car."""
        try:
            response = requests.get(link, headers=self.headers)
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'lxml')
                car_data = parse_bonbanh_car_details(soup, brand=brand or self.current_brand)
                car_data['url'] = link
                return car_data
        except Exception as e:
            print(f"Error: {e}")
        return None
    
    def _append_to_csv(self, car_data: Dict, filepath: Path) -> bool:
        """Append single car to CSV immediately with data cleaning. Returns True if added, False if duplicate."""
        try:
            car_url = car_data.get('url')
            
            # Check for duplicate URL
            with self.csv_lock:
                if car_url in self.existing_urls:
                    return False  # Skip duplicate
                
                car_data.pop('crawl_timestamp', None)
                car_data.pop('title', None)
                
                # Clean and format data before saving
                cleaned_data = clean_car_data(car_data)
                df = pd.DataFrame([cleaned_data])
                
                if filepath.exists():
                    df.to_csv(filepath, mode='a', header=False, index=False, encoding='utf-8-sig')
                else:
                    df.to_csv(filepath, mode='w', header=True, index=False, encoding='utf-8-sig')
                
                # Add URL to set after successful write
                self.existing_urls.add(car_url)
                return True
        except Exception as e:
            print(f"CSV error: {e}")
            return False
    
    def crawl_brand(self, brand: str, max_cars: int = 500, max_pages: int = None, save_to_csv: bool = True):
        """Crawl cars of a specific brand."""
        self.current_brand = brand
        cars_crawled = 0
        
        # Setup CSV
        output_path = Path("../data")
        output_path.mkdir(parents=True, exist_ok=True)
        csv_filepath = output_path / "car_data.csv"
        
        # Load existing URLs to prevent duplicates
        if csv_filepath.exists():
            try:
                existing_df = pd.read_csv(csv_filepath)
                self.existing_urls = set(existing_df['url'].dropna().tolist())
                print(f"📋 Loaded {len(self.existing_urls)} existing URLs")
            except Exception as e:
                print(f"⚠ Could not load existing URLs: {e}")
                self.existing_urls = set()
        
        # Check progress
        if self.progress.is_completed('bonbanh', brand):
            print(f"⚠ Brand '{brand}' already completed")
            return cars_crawled
        
        page = self.progress.get_last_page('bonbanh', brand)
        if page > 0:
            print(f"📍 Resuming from page {page + 1}")
        
        print(f"\n{'='*60}")
        print(f"Crawling {brand} - Max: {max_cars} cars")
        print(f"CSV: {'Enabled' if save_to_csv else 'Disabled'}")
        print(f"{'='*60}\n")
        
        while True:
            if max_pages and page >= max_pages:
                break
            if max_cars and cars_crawled >= max_cars:
                break
            
            try:
                url = f'{self.BASE_URL}{brand}/page,{page}'
                print(f"Page {page + 1}: {url}")
                
                response = requests.get(url, headers=self.headers)
                if response.status_code == 200:
                    soup = BeautifulSoup(response.content, 'lxml')
                    car_links = parse_bonbanh_listing(soup)
                    
                    if not car_links:
                        print("No more cars found")
                        break
                    
                    print(f"Found {len(car_links)} cars")
                    
                    for link in car_links:
                        if max_cars and cars_crawled >= max_cars:
                            break
                        
                        car_data = self.crawl_car_details(link, brand=brand)
                        if car_data:
                            if save_to_csv:
                                if self._append_to_csv(car_data, csv_filepath):
                                    cars_crawled += 1
                                    print(f"  [{cars_crawled}] ✓ {car_data.get('model', 'Unknown')}")
                                else:
                                    print(f"  [SKIP] Duplicate URL")
                            else:
                                cars_crawled += 1
                                print(f"  [{cars_crawled}] ✓ {car_data.get('model', 'Unknown')}")
                        
                        rate_limit(self.delay)
                    
                    self.progress.update_page('bonbanh', page, brand)
                    page += 1
                else:
                    print(f"Failed: {response.status_code}")
                    break
            except Exception as e:
                print(f"Error: {e}")
                break
        
        print(f"\n{'='*60}")
        print(f"Completed: {cars_crawled} cars")
        print(f"{'='*60}\n")
        
        if not (max_cars and cars_crawled >= max_cars):
            self.progress.mark_completed('bonbanh', brand)
        
        return cars_crawled

print("✓ BonbanhCrawler class defined")

✓ BonbanhCrawler class defined


## 7. Chotot Crawler

In [29]:
class ChototCrawler:
    """Crawler for Chotot.com car listings with multi-threaded queue-based processing."""
    
    BASE_URL = "https://xe.chotot.com/mua-ban-oto"
    
    def __init__(self, delay: float = 0.5, max_workers: int = 5):
        self.headers = get_headers()
        self.delay = delay
        self.max_workers = max_workers
        self.csv_lock = threading.Lock()
        self.counter_lock = threading.Lock()
        self.progress = ProgressTracker()
        self.existing_urls = set()  # Track URLs to prevent duplicates
    
    def crawl_car_details(self, link: str) -> Optional[Dict[str, str]]:
        """Crawl details of a single car."""
        try:
            response = requests.get(link, headers=self.headers)
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'lxml')
                car_data = parse_chotot_car_details(soup)
                car_data['url'] = link
                return car_data
        except Exception as e:
            print(f"Error crawling {link}: {e}")
        return None
    
    def _append_to_csv(self, car_data: Dict, filepath: Path) -> bool:
        """Append single car to CSV immediately (thread-safe) with data cleaning. Returns True if added, False if duplicate."""
        try:
            car_url = car_data.get('url')
            
            # Check for duplicate URL
            with self.csv_lock:
                if car_url in self.existing_urls:
                    return False  # Skip duplicate
                
                car_data.pop('crawl_timestamp', None)
                car_data.pop('title', None)
                
                # Clean and format data before saving
                cleaned_data = clean_car_data(car_data)
                df = pd.DataFrame([cleaned_data])
                
                if filepath.exists():
                    df.to_csv(filepath, mode='a', header=False, index=False, encoding='utf-8-sig')
                else:
                    df.to_csv(filepath, mode='w', header=True, index=False, encoding='utf-8-sig')
                
                # Add URL to set after successful write
                self.existing_urls.add(car_url)
                return True
        except Exception as e:
            print(f"CSV error: {e}")
            return False
    
    def _process_car_link(self, link: str, csv_filepath: Path, save_to_csv: bool) -> bool:
        """Worker function to process a single car link from queue."""
        car_data = self.crawl_car_details(link)
        if car_data:
            if save_to_csv:
                return self._append_to_csv(car_data, csv_filepath)
            rate_limit(self.delay)
            return True
        return False
    
    def crawl_listings(self, max_cars: int = 5000, max_pages: int = None, save_to_csv: bool = True):
        """Crawl car listings from Chotot using queue-based multi-threading."""
        cars_crawled = 0
        
        # Setup CSV
        output_path = Path("../data")
        output_path.mkdir(parents=True, exist_ok=True)
        csv_filepath = output_path / "car_data.csv"
        
        # Load existing URLs to prevent duplicates
        if csv_filepath.exists():
            try:
                existing_df = pd.read_csv(csv_filepath)
                self.existing_urls = set(existing_df['url'].dropna().tolist())
                print(f"📋 Loaded {len(self.existing_urls)} existing URLs")
            except Exception as e:
                print(f"⚠ Could not load existing URLs: {e}")
                self.existing_urls = set()
        
        # Check progress
        if self.progress.is_completed('chotot'):
            print("⚠ Chotot already completed")
            return cars_crawled
        
        page = self.progress.get_last_page('chotot')
        if page > 0:
            page += 1
            print(f"📍 Resuming from page {page}")
        else:
            page = 1
        
        print(f"\n{'='*60}")
        print(f"Crawling Chotot.com - Max: {max_cars} cars")
        print(f"Multi-threading: {self.max_workers} workers")
        print(f"CSV: {'Enabled' if save_to_csv else 'Disabled'}")
        print(f"{'='*60}\n")
        
        while True:
            if max_pages and page > max_pages:
                break
            if max_cars and cars_crawled >= max_cars:
                break
            
            try:
                url = f'{self.BASE_URL}?page={page}'
                print(f"Page {page}: {url}")
                
                response = requests.get(url, headers=self.headers)
                if response.status_code == 200:
                    soup = BeautifulSoup(response.content, 'lxml')
                    car_links = parse_chotot_listing(soup)
                    
                    if not car_links:
                        print("No more cars found")
                        break
                    
                    print(f"Found {len(car_links)} cars on page {page}")
                    
                    # Calculate how many cars to process from this page
                    remaining_quota = max_cars - cars_crawled if max_cars else len(car_links)
                    links_to_process = car_links[:remaining_quota]
                    
                    # Process car links in parallel using ThreadPoolExecutor
                    with concurrent.futures.ThreadPoolExecutor(max_workers=self.max_workers) as executor:
                        # Submit all car link processing tasks
                        future_to_link = {
                            executor.submit(self._process_car_link, link, csv_filepath, save_to_csv): link 
                            for link in links_to_process
                        }
                        
                        # Collect results as they complete
                        page_crawled = 0
                        for future in concurrent.futures.as_completed(future_to_link):
                            link = future_to_link[future]
                            try:
                                success = future.result()
                                if success:
                                    with self.counter_lock:
                                        cars_crawled += 1
                                        page_crawled += 1
                                    print(f"  [{cars_crawled}] ✓ Crawled")
                            except Exception as e:
                                print(f"  ✗ Error processing {link}: {e}")
                    
                    print(f"Page {page} completed: {page_crawled}/{len(links_to_process)} cars processed")
                    
                    self.progress.update_page('chotot', page)
                    page += 1
                    
                    # Check if we've reached the max_cars limit
                    if max_cars and cars_crawled >= max_cars:
                        break
                else:
                    print(f"Failed: {response.status_code}")
                    break
            except Exception as e:
                print(f"Error: {e}")
                break
        
        print(f"\n{'='*60}")
        print(f"Completed: {cars_crawled} cars")
        print(f"{'='*60}\n")
        
        if not (max_cars and cars_crawled >= max_cars):
            self.progress.mark_completed('chotot')
        
        return cars_crawled

print("✓ ChototCrawler class defined")

✓ ChototCrawler class defined


## 8. Usage Examples

### Example 1: Crawl Single Brand (Bonbanh)

In [30]:
# # Crawl Toyota cars from Bonbanh 
# bonbanh_crawler = BonbanhCrawler(delay=1.0)
# cars_count = bonbanh_crawler.crawl_brand(
#     brand='toyota',
#     max_cars=10  # Change to 500 for full crawl
# )

# print(f"\nTotal cars crawled and saved: {cars_count}")

### Example 2: Crawl Chotot

In [31]:
# # Crawl cars from Chotot
# chotot_crawler = ChototCrawler(delay=0.5)
# cars_count = chotot_crawler.crawl_listings(
#     max_cars=10  # Change to 5000 for full crawl
# )

# print(f"\nTotal cars crawled and saved: {cars_count}")

### Example 3: Crawl Multiple Brands with Threading

In [32]:
# def crawl_single_brand(brand: str, max_cars: int = 30):
#     """Helper function for parallel crawling."""
#     crawler = BonbanhCrawler(delay=0.0)
#     return crawler.crawl_brand(brand, max_cars=max_cars)

# # Crawl multiple brands in parallel 
# brands_to_crawl = CAR_BRANDS

# with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
#     future_to_brand = {
#         executor.submit(crawl_single_brand, brand, 5): brand 
#         for brand in brands_to_crawl
#     }
    
#     total_cars = 0
#     for future in concurrent.futures.as_completed(future_to_brand):
#         brand = future_to_brand[future]
#         try:
#             count = future.result()
#             total_cars += count
#             print(f"✓ {brand}: {count} cars saved")
#         except Exception as e:
#             print(f"✗ {brand}: {e}")

# print(f"\nTotal cars crawled: {total_cars}")

## 9. Data Exploration

In [33]:
# # Load and explore crawled data
# bonbanh_df = pd.read_csv('../data/bonbanh.csv')
# chotot_df = pd.read_csv('../data/chotot.csv')

# print("Bonbanh Data:")
# print(f"Total records: {len(bonbanh_df)}")
# print(bonbanh_df.head())
# print("\nColumn info:")
# print(bonbanh_df.info())

# print("\n" + "="*60)
# print("Chotot Data:")
# print(f"Total records: {len(chotot_df)}")
# print(chotot_df.head())
# print("\nColumn info:")
# print(chotot_df.info())

## 10. Progress Management

In [34]:
# # View current progress
# progress = ProgressTracker()

# # Show all progress
# print("Current Progress:")
# print(json.dumps(progress.progress_data, indent=2, ensure_ascii=False))

# # Reset specific brand
# # progress.reset('bonbanh', 'toyota')

# # Reset all progress
# # progress.reset()

## 11. Full Production Crawl

In [35]:
# WARNING: This will crawl ALL brands with default limits
# Bonbanh: 500 cars per brand * 29 brands (Some brands may not have enough 500 cars)
# Chotot: 5,000 cars
# Multi-threading: Crawls multiple brands in parallel for faster execution

# Helper function for parallel brand crawling
def crawl_brand_worker(brand: str, max_cars: int = 500):
    """Worker function for parallel crawling."""
    crawler = BonbanhCrawler(delay=0.0)
    return brand, crawler.crawl_brand(brand, max_cars=max_cars)

# Crawl all Bonbanh brands with multi-threading
print(f"\n{'='*80}")
print(f"STARTING FULL CRAWL - {len(CAR_BRANDS)} BRANDS")
print(f"{'='*80}\n")

total_cars_bonbanh = 0
max_workers = 5  # Number of parallel threads (adjust based on your system)

with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Submit all brand crawling tasks
    future_to_brand = {
        executor.submit(crawl_brand_worker, brand, 10): brand 
        for brand in CAR_BRANDS
    }
    
    # Process completed tasks as they finish
    for i, future in enumerate(concurrent.futures.as_completed(future_to_brand), 1):
        brand = future_to_brand[future]
        try:
            brand_name, count = future.result()
            total_cars_bonbanh += count
            print(f"\n[{i}/{len(CAR_BRANDS)}] ✓ {brand_name.upper()}: {count} cars")
        except Exception as e:
            print(f"\n[{i}/{len(CAR_BRANDS)}] ✗ {brand.upper()}: Error - {e}")

print(f"\n{'='*80}")
print(f"BONBANH COMPLETED: {total_cars_bonbanh} total cars from {len(CAR_BRANDS)} brands")
print(f"{'='*80}\n")

# Crawl Chotot (single-threaded as it's one source)
chotot_crawler = ChototCrawler(delay=0.0, max_workers=5)
total_cars_chotot = chotot_crawler.crawl_listings(max_cars=50)

print(f"\n{'='*80}")
print(f"FULL CRAWL COMPLETED")
print(f"  Bonbanh: {total_cars_bonbanh} cars")  
print(f"  Chotot: {total_cars_chotot} cars")
print(f"  TOTAL: {total_cars_bonbanh + total_cars_chotot} cars")
print(f"{'='*80}\n")


STARTING FULL CRAWL - 29 BRANDS


Crawling toyota - Max: 10 cars
CSV: Enabled

Page 1: https://bonbanh.com/oto/toyota/page,0

Crawling hyundai - Max: 10 cars
CSV: Enabled

Page 1: https://bonbanh.com/oto/hyundai/page,0

Crawling kia - Max: 10 cars
CSV: Enabled

Page 1: https://bonbanh.com/oto/kia/page,0

Crawling ford - Max: 10 cars
CSV: Enabled

Page 1: https://bonbanh.com/oto/ford/page,0

Crawling honda - Max: 10 cars
CSV: Enabled

Page 1: https://bonbanh.com/oto/honda/page,0
No more cars found

Completed: 0 cars


[1/29] ✓ HYUNDAI: 0 cars

Crawling mazda - Max: 10 cars
CSV: Enabled

Page 1: https://bonbanh.com/oto/mazda/page,0
No more cars found

Completed: 0 cars


[2/29] ✓ HONDA: 0 cars

Crawling mitsubishi - Max: 10 cars
CSV: Enabled

Page 1: https://bonbanh.com/oto/mitsubishi/page,0
No more cars found

Completed: 0 cars


[3/29] ✓ TOYOTA: 0 cars
No more cars found

Completed: 0 cars


[4/29] ✓ KIA: 0 cars
No more cars found

Completed: 0 cars


Crawling vinfast - Max: 10 cars
C